In [13]:
import math
import os
import shutil

In [14]:
def parse_problem(problem_file):

    x1_obs = []
    x4_obs = []
    x7_obs = []
    t_obs = []

    with open(problem_file, "r") as f:
        for line in f.readlines():
            if "(time_obs" in line:
                t_obs += [float(line.strip().replace("(", "").replace(")", "").split(" ")[3])]
            elif "(x1_obs" in line:
                x1_obs += [float(line.strip().replace("(", "").replace(")", "").split(" ")[3])]
            elif "(x4_obs" in line:
                x4_obs += [float(line.strip().replace("(", "").replace(")", "").split(" ")[3])]
            elif "(x7_obs" in line:
                x7_obs += [float(line.strip().replace("(", "").replace(")", "").split(" ")[3])]


    observations = list(zip(x1_obs, x4_obs, x7_obs, t_obs))
    
    return observations

In [40]:
def define_ha(observations):
    
    num_obs = len(observations)
    lower_bound = -200
    upper_bound = 200
    
    ha_str = ""
        
    # Variables
    ha_str += "[0,{}] time;\n".format(observations[-1][3] + 0.05)
    ha_str += "[{},{}] x1;\n".format(lower_bound,upper_bound)
    ha_str += "[{},{}] x2;\n".format(lower_bound,upper_bound)
    ha_str += "[{},{}] x3;\n".format(lower_bound,upper_bound)
    ha_str += "[{},{}] x4;\n".format(lower_bound,upper_bound)
    ha_str += "[{},{}] x5;\n".format(lower_bound,upper_bound)
    ha_str += "[{},{}] x6;\n".format(lower_bound,upper_bound)
    ha_str += "[{},{}] x7;\n".format(lower_bound,upper_bound)
    ha_str += "[{},{}] x8;\n".format(lower_bound,upper_bound)
    ha_str += "[{},{}] x9;\n".format(lower_bound,upper_bound)
    ha_str += "[0,{}] clock;\n".format(observations[-1][3] + 0.05)
    ha_str += "\n"
    
    # Modes
    modes = {}
    num_modes = 0
    for hmode in ["elf", "dreizehn"]:
        for obs_index in range(num_obs+1):
            modes[(hmode,obs_index)] = num_modes + 1
            num_modes += 1
    
    for hmode in ["elf", "dreizehn"]:
        if hmode == "elf":
            dst_offset = num_obs + 1
        else:
            dst_offset = -num_obs - 1 
            
        for obs_index in range(num_obs+1):
            mode_id = modes[(hmode, obs_index)]
            ha_str += "// {}_obs{}\n".format(hmode,obs_index)
            ha_str += "{\n"
            ha_str += "\t mode {};\n".format(mode_id)
            # Invariant
            ha_str += "\t invt:\n"
            if obs_index < num_obs:
                ha_str += "\t\t (clock < {} + 0.05);\n".format(observations[obs_index][3])
            else:
                ha_str += "\t\t (clock < {} + 0.05);\n".format(observations[-1][3])
            # Flow
            ha_str += "\t flow:\n"
            if hmode == "elf":
                ha_str += "\t\t d/dt[x1] = x2;\n"
                ha_str += "\t\t d/dt[x2] = -x3 + -4.0;\n"
                ha_str += "\t\t d/dt[x3] = 1.605 * x1 + 4.868 * x2 - 3.5754 * x3 - 0.8198 * x4 + 0.427 * x5 - 0.045 * x6 - 0.1942 * x7 + 0.3626 * x8 - 0.0946 * x9;\n"
                ha_str += "\t\t d/dt[x4] = x5;\n"
                ha_str += "\t\t d/dt[x5] = x3 - x6;\n"
                ha_str += "\t\t d/dt[x6] = 0.8718 * x1 + 3.814 * x2 - 0.0754 * x3 + 1.1936 * x4 + 3.6258 * x5 - 3.2396 * x6 - 0.595 * x7 + 0.1294 * x8 - 0.0796 * x9;\n"
                ha_str += "\t\t d/dt[x7] = x8;\n"
                ha_str += "\t\t d/dt[x8] = x6 - x9;\n"
                ha_str += "\t\t d/dt[x9] = 0.7132 * x1 + 3.573 * x2 - 0.0964 * x3 + 0.8472 * x4 + 3.2568 * x5 - 0.0876 * x6 + 1.2726 * x7 + 3.072 * x8 - 3.1356 * x9;\n"
            else:
                ha_str += "\t\t d/dt[x1] = x2;\n"
                ha_str += "\t\t d/dt[x2] = -x3 + -4.0;\n"
                ha_str += "\t\t d/dt[x3] = 1.605 * x1 + 4.868 * x2 - 3.5754 * x3;\n"
                ha_str += "\t\t d/dt[x4] = x5;\n"
                ha_str += "\t\t d/dt[x5] = x3 - x6;\n"
                ha_str += "\t\t d/dt[x6] = 1.1936 * x4 + 3.6258 * x5 - 3.2396 * x6;\n"
                ha_str += "\t\t d/dt[x7] = x8;\n"
                ha_str += "\t\t d/dt[x8] = x6 - x9;\n"
                ha_str += "\t\t d/dt[x9] = 0.7132 * x1 + 3.573 * x2 - 0.0964 * x3 + 0.8472 * x4 + 3.2568 * x5 - 0.0876 * x6 + 1.2726 * x7 + 3.072 * x8 - 3.1356 * x9;\n"
                
            ha_str += "\t\t d/dt[clock] = 1.0;\n"
            
            # Jumps
            
            ha_str += "\t jump:\n"
            ha_str += "\t\t (true) ==> @{}(and (x1' = x1) (x2' = x2) (x3' = x3) (x4' = x4) (x5' = x5) (x6' = x6) (x7' = x7) (x8' = x8) (x9' = x9) (clock' = clock));\n".format(mode_id + dst_offset)
            if obs_index < num_obs:    
                ha_str += "\n"
                ha_str += "\t\t (and (and (and (clock > {clock_obs} - 0.05) (clock < {clock_obs} + 0.05)) (and (x1 > {x1_obs} - 0.1) (x1 < {x1_obs} + 0.1))) (and (and (x4 > {x4_obs} - 0.1) (x4 < {x4_obs} + 0.1)) (and (x7 > {x7_obs} - 0.1) (x7 < {x7_obs} + 0.1)))) ==> @{dst_id}(and (x1' = x1) (x2' = x2) (x3' = x3) (x4' = x4) (x5' = x5) (x6' = x6) (x7' = x7) (x8' = x8) (x9' = x9) (clock' = clock));\n".format(clock_obs = observations[obs_index][3], x1_obs = observations[obs_index][0], x4_obs = observations[obs_index][1], x7_obs = observations[obs_index][2], dst_id = mode_id+1)
                
            ha_str += "}\n"
            
    # Init
    ha_str += "init:\n"
    ha_str += "@1 (and (and (and (x1 = 0.0) (x2 = 0.0)) (and (x3 = 0.0) (and (x4 = 0.0) (x5 = 0.0)))) (and (and (x6 = 0.0) (x7 = 0.0)) (and (x8 = 0.0) (and (x9 = 0.0) (clock = 0.0)))));\n"
    ha_str += "\n"
    
    # Goal
    ha_str += "goal:\n"
    for hmode in ["elf", "dreizehn"]:
        mode_id = modes[(hmode, num_obs)]
        ha_str += "@{} true;\n".format(mode_id)
            
    return ha_str   

In [41]:
num_instances = 10
horizons = [2, 4, 6, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

dfolder = "benchmarks/platoon/dreach"
if os.path.exists(dfolder):
    shutil.rmtree(dfolder)
os.makedirs(dfolder)


for pnum in range(num_instances):
    for h in horizons:
        pname = 'inst_{pnum}_{h}.drh'.format(pnum=pnum, h=h)
        
        observations = parse_problem("benchmarks/platoon/generation/inst_{}_{}_5_100.pddl".format(pnum, h))

        ha_str = define_ha(observations)
        with open(dfolder + '/' + pname, 'w') as f:
            f.write(ha_str)